In [12]:
# Imports
import scipy as sp
from scipy import linalg
import numpy as np
from numpy import random
import xlsxwriter

import openpyxl
from openpyxl import load_workbook

import pandas as pd

In [13]:
# OPENPYXL TESTING

# workbook = load_workbook(filename=r'C:\Users\Armin\Desktop\EE454\Project\Three Bus System Example\system_SampleInput.xlsx')
# busData = workbook['BusData']
# lineData = workbook['LineData']

In [25]:
# Creates bus data matrix
S_base = 100.
max_mismatch = 0.1

dfBus = pd.read_excel (r'C:\Users\Armin\Desktop\EE454\Project\Three Bus System Example\system_SampleInput.xlsx', sheet_name='BusData')
dfLine = pd.read_excel (r'C:\Users\Armin\Desktop\EE454\Project\Three Bus System Example\system_SampleInput.xlsx', sheet_name='LineData')

busData = dfBus.values
lineData = dfLine.values

busRows = len(busData)
busCols = len(busData[0])
lineRows = len(lineData)
lineCols = len(lineData[0])

print(dfBus)
print(dfLine)

   Bus #  P MW  Q MVAr Type  P Gen  V Set
0      1     0       0    S      0   1.00
1      2    45      15    G     25   1.02
2      3    53       7    D      0   1.00
   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
0     1   2       0.08205       0.13027        0.0528      99999
1     1   3       0.12291       0.25202        0.0128      99999
2     2   3       0.22092       0.19988        0.0000      99999


In [27]:
# Creates admittance matrix
Y = 1j * np.zeros((busRows, busRows))
G = np.zeros((busRows, busRows))
B = np.zeros((busRows, busRows))

B_cap = {}


for index, row in dfLine.iterrows():
#     print(row)
#     print("{} {}".format(row['Rtotal, p.u.'], row['Xtotal, p.u.']))
    Y[int(row['From']) - 1][int(row['To']) - 1] = -(row['Rtotal, p.u.'] + 1j * row['Xtotal, p.u.'])**-1
    Y[int(row['To']) - 1][int(row['From']) - 1] = -(row['Rtotal, p.u.'] + 1j * row['Xtotal, p.u.'])**-1
    B_cap[int(row['From'])] = B_cap.get(row['From'], []) + [row['Btotal, p.u.']]
    B_cap[int(row['To'])] = B_cap.get(row['To'], []) + [row['Btotal, p.u.']]
   
for i in range(busRows):
    Y[i][i] = -sum([Y[i][x] for x in range(busRows)]) + 0.5j*sum(B_cap[i + 1])
    
G = Y.real
B = Y.imag

print(B)


[[-8.66875742  5.49605043  3.20550699]
 [ 5.49605043 -7.72162505  2.25197463]
 [ 3.20550699  2.25197463 -5.45108162]]


In [75]:
# Creates load matrix
Load_map = {}
Load_matrix = np.zeros((3,busRows))

for i in range(busRows):
    load[0][i] = busData[i][0]
    load[1][i] = busData[i][1]
    load[2][i] = busData[i][2]
    
print(load)


[[ 1.  2.  3.]
 [ 0. 45. 53.]
 [ 0. 15.  7.]]


In [77]:
# Active power matrix
Gen_map = {}
Gen_matrix = np.zeros((2,busRows))

for index, row in dfBus.iterrows():
    Gen_map[(row['Bus #'])] = B_cap.get(row['Type'], []) + [row['P Gen']]

for i in range(busRows):
    Gen_matrix[0][i] = busData[i][0]
    if busData[i][3] == 'G':
        Gen_matrix[1][i] = busData[i][4]

print(Gen_matrix)

[[ 1.  2.  3.]
 [ 0. 25.  0.]]


In [79]:
# Reference voltages at bus
v_bus = np.zeros((2,busRows))

for i in range(busRows):
    v_bus[0][i] = busData[i][0]
    v_bus[1][i] = busData[i][5]

print(v_bus)

[[1.   2.   3.  ]
 [1.   1.02 1.  ]]


In [ ]:
# Power flow solving
for i in range(busRows)
    

In [80]:
# Power flow function
def calcs(P_gen, P_load, Q_gen, Q_load)
    theta_flat = 0
    v_flat = 1
    P_inj = P_gen - P_load
    Q_inj = Q_gen - Q_load